In [1]:
from fomlads.evaluate.eval_regression import train_and_test_partition
from fomlads.evaluate.eval_regression import train_and_test_filter
from fomlads.evaluate.eval_regression import create_cv_folds

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./Data/Used Dataset.csv')
df

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,track_genre
0,44,0.00621,0.711,285987,0.6210,0.029700,11,0.159,-7.429,1,0.0382,142.065,0.607,Jazz
1,34,0.34700,0.729,103703,0.5150,0.006920,4,0.127,-10.147,1,0.1840,79.969,0.287,Jazz
2,39,0.91800,0.484,228733,0.0693,0.000069,9,0.139,-15.542,1,0.0346,82.707,0.206,Jazz
3,36,0.03450,0.473,342867,0.8230,0.446000,11,0.306,-6.625,1,0.0630,104.694,0.383,Jazz
4,53,0.50100,0.921,151733,0.2660,0.134000,2,0.109,-12.749,0,0.2130,123.043,0.761,Jazz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22583,59,0.03340,0.913,-1,0.5740,0.000000,5,0.119,-7.022,1,0.2980,98.028,0.330,Hip-Hop
22584,72,0.15700,0.709,251860,0.3620,0.000000,3,0.109,-9.814,1,0.0550,122.043,0.113,Hip-Hop
22585,51,0.00597,0.693,189483,0.7630,0.000000,6,0.143,-5.443,1,0.1460,131.079,0.395,Hip-Hop
22586,65,0.08310,0.782,262773,0.4720,0.000000,11,0.106,-5.016,0,0.0441,75.886,0.354,Hip-Hop


In [3]:
# FUNCTION 1: Function separates inputs and targets in the dataset

def inputs_and_targets(pdDataset):
    number_of_columns = len(pdDataset.axes[1])
    npDataset = pdDataset.to_numpy() # Convert dataset to a numpy array
    inputs = npDataset[: , :(number_of_columns - 1)] # Split numpy array since genres/targets are only in the last column
    targets = npDataset[: , (number_of_columns - 1)].astype('object')
    return inputs, targets

inputs_0, targets_0 = inputs_and_targets(df)

In [36]:
np.random.seed(1) # Ensure the same random number occurs after rerunning
shuffle_index = np.random.permutation(len(inputs_0)) # Shuffle the index of the dataset
shuffle_index

array([ 6353,  9861,  5181, ...,  5192, 12172,   235])

In [37]:
train_per = 0.7
train_num = int(train_per * len(inputs_0))
print(train_num)

15811


In [41]:
train_index = shuffle_index[:train_num]
validation_index = shuffle_index[train_num:]

train_inputs = inputs_0[train_index]
train_targets = targets_0[train_index]
validation_inputs = inputs_0[validation_index]
validation_targets = targets_0[validation_index]

In [42]:
print(f"train_inputs = {train_inputs}")
print(f"train_targets = {train_targets}")
print(f"validation_inputs = {validation_inputs}")
print(f"validation_targets = {validation_targets}")

train_inputs = [[39 0.0211 0.479 ... 0.0287 158.231 0.917]
 [59 0.00208 0.421 ... 0.0338 171.814 0.438]
 [41 0.0514 0.484 ... 0.0314 74.651 0.728]
 ...
 [45 0.894 0.508 ... 0.0374 103.993 0.399]
 [35 0.908 0.305 ... 0.0355 154.775 0.24]
 [56 0.817 0.556 ... 0.0268 109.586 0.764]]
train_targets = ['Country' 'Rock' 'Country' ... 'Classical' 'Classical' 'Rock']
validation_inputs = [[40 0.926 0.366 ... 0.0316 78.92 0.209]
 [57 0.0923 0.789 ... 0.0456 104.593 0.826]
 [57 0.00721 0.797 ... 0.0404 99.828 0.948]
 ...
 [53 0.722 0.607 ... 0.0269 134.003 0.536]
 [60 0.0326 0.545 ... 0.0491 110.008 0.727]
 [57 0.725 0.726 ... 0.0391 127.994 0.656]]
validation_targets = ['Classical' 'Rock' 'Hip-Hop' ... 'Country' 'Rock' 'Jazz']


In [43]:
# FUNCTION 2: Function calculates accuracy score by comparing the actual and predicted targets.

def accuracyscore(actual_target, predict_target):
    n = len(actual_target)
    score = 0
    for i in range(n):
        if actual_target[i] == predict_target[i]: # Calculate how many targets are correctly predicted
            score += 1
    accuracy = score / n # Divide number of correctly predicted targets by total for accuracy score
    return accuracy

In [44]:
# FUNCTION 3: Function takes an array and standardises the columns.

def standardised_array(array):
    standard_array = array.copy() # Make a copy of the array and store it as the standardised array
    for i in range(array.shape[1]): # Standardise by going through each column
        mean = array[: , i].mean()
        std = array[: , i].std()
        standard_array[: , i] = (array[: , i] - mean) / std
    return standard_array